# Managing features

In [ ]:
from pycellin.io.trackmate import load_TrackMate_XML

In [ ]:
# Path to the TrackMate XML file.
xml_path = "../sample_data/FakeTracks.xml"

# Parse the XML file and create a Pycellin Model object
# that contains all the data from the XML file.
model = load_TrackMate_XML(xml_path)

# We can display basic information about this model.
print(model)
print(f"This model contains {model.data.number_of_lineages()} lineages:")
for lin_ID, lineage in model.data.cell_data.items():
    print(f"- ID {lin_ID}: {lineage}")

## Imported features

When the data was created wih an external tool, e.g. TrackMate.
Cannot be recomputed.

## Pycellin features

## Custom features

### How to choose which calculator to use?

Tree diagram of FeatureCalculator classes